In [1]:
#import tensorflow as tf
import numpy as np
#import keras_cv
import cv2
#from tensorflow import keras
from matplotlib import pyplot as plt
import carla
#import time
from ultralytics import YOLO
import easyocr
import sys
import os

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()
model_input_shape = 1024

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def resize_image(image):
    if max(image.shape[0], image.shape[1]) > model_input_shape:
        if image.shape[0] > image.shape[1]:
            height = model_input_shape
            width = int(image.shape[1] * model_input_shape / image.shape[0])
        else:
            height = int(image.shape[0] * model_input_shape / image.shape[1])
            width = model_input_shape
    resized_image = cv2.resize(image, (int(width), int(height)), cv2.INTER_AREA)
    resized_image = cv2.copyMakeBorder(resized_image, 0, model_input_shape - resized_image.shape[0], 0, model_input_shape - resized_image.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0])
    scale = model_input_shape / max(image.shape[0], image.shape[1])
    return scale, resized_image

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '45')
camera_bp.set_attribute('sensor_tick', '0.0')
camera_bp.set_attribute('motion_blur_intensity', '0.0')
spawn_point = carla.Transform()

#yolov8_backbone = keras_cv.models.YOLOV8Backbone.from_preset('yolo_v8_s_backbone_coco',load_weights=True)
yolov8_plates_detector = YOLO("yolo_pytorch/best.pt")

#yolov8_plates_detector = keras_cv.models.YOLOV8Detector(backbone=yolov8_backbone,num_classes=1,bounding_box_format='xywh')
#yolov8_plates_detector.compile(optimizer='adam', classification_loss='binary_crossentropy', box_loss='ciou')
#yolov8_plates_detector.load_weights("model_checkpoints_3.8/checkpoint_epoch_24.weights.h5")
reader = easyocr.Reader(["en"], gpu=True)

In [14]:
ego_vehicle = spawn_vehicle(spawn_index=7)
static_vehicle = spawn_vehicle(spawn_index=9)
running = True

camera = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.7, z=1.2), carla.Rotation(pitch=-10)), attach_to=ego_vehicle)
video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)
saved_location = carla.Location(x=0, y=0, z=0)
#figure = plt.figure(figsize=(2, 1))

sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])


def camera_callback(image):
    global video_output
    #output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    image_bgr = cv2.cvtColor(image_array, cv2.COLOR_BGRA2BGR)
    
    scale, resized_image = resize_image(image_bgr)
    #sys.stdout = open(os.devnull, 'w')
    pred = yolov8_plates_detector.predict(resized_image)
    #sys.stdout = sys.__stdout__
    rescaling_factor = 1 / scale
    for result in pred:
        print("Result: " + str(result))
        boxes = result.boxes
        if boxes:
            for box in boxes:
                plate = image_bgr[int(box.xyxy[0][1].item() * rescaling_factor) : int(box.xyxy[0][3].item() * rescaling_factor), int(box.xyxy[0][0].item() * rescaling_factor) : int(box.xyxy[0][2].item() * rescaling_factor)]
                plate_enhanced = cv2.filter2D(plate, -1, sharpen_kernel)
                plate_reading = reader.readtext(plate_enhanced)
                #plt.imshow(plate)
                #plt.axis("off")
                #plt.show()
                cv2.rectangle(image_bgr, (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), (int((box.xyxy[0][2].item() * rescaling_factor)), int((box.xyxy[0][3].item() * rescaling_factor))), (0, 255, 0), 1)
                read_flag = False
                for plate_detected in plate_reading:
                    confidence = plate_detected[2]
                    plate_text = plate_detected[1]
                    print(plate_text)
                    if confidence > 0.5:
                        read_flag = True
                        cv2.putText(image_bgr, plate_text, (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                if not read_flag:
                    cv2.putText(image_bgr, "????", (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                        
    video_output = image_bgr


camera.listen(lambda image: camera_callback(image))

ego_vehicle_controls = carla.VehicleControl()
ego_vehicle_controls.throttle = 0.2
ego_vehicle.apply_control(ego_vehicle_controls)
#ego_vehicle.set_autopilot(True)
#static_vehicle.set_autopilot(True)
print(static_vehicle.attributes.items)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)



try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        if video_output is not None:
            cv2.imshow('RGB Camera', video_output)
        if cv2.waitKey(1) == ord('s'):
            saved_location = ego_vehicle.get_location()
            print("Saved location: ", saved_location)
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    print("Shutting down...")
    cv2.destroyAllWindows()
    if 'camera' in locals() and camera.is_alive:
        camera.destroy()
    for actor in world.get_actors():
        if actor.type_id.startswith('vehicle.') or actor.type_id.startswith('sensor.camera'):
            actor.destroy()


In [37]:
for vehicle in world.get_actors().filter('vehicle.*'):
    vehicle.destroy()


In [50]:
world.set_weather(carla.WeatherParameters.ClearNight)

In [10]:
import torch, torchvision
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torchvision.__version__)


In [2]:
blueprint_library = world.get_blueprint_library()

# Filter vehicle blueprints
vehicles_with_plates = []

for blueprint in blueprint_library.filter('vehicle.*'):
    try:
        # Check if the blueprint has a 'license_plate' attribute
        if blueprint.has_attribute('license_plate'):
            vehicles_with_plates.append(blueprint.id)
    except Exception as e:
        print(f"Error checking blueprint {blueprint.id}: {e}")

# Print vehicles that support license plate changes
print("Vehicles with customizable license plates:")
for vehicle_id in vehicles_with_plates:
    print(vehicle_id)
print(len(vehicles_with_plates))